# Рекомендательные системы

Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

In [361]:
import pandas as pd
from collections import Counter, OrderedDict
import numpy as np

In [362]:
# сохраним ответ в блокнот
def save_answers(k, s):
    with open(s, "w") as fout:
        fout.write(str(k))

In [363]:
def M(l):
    n = []
    for i in l:
        if i not in n:
            n.append(i)
    return n

Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
 сортировка просмотренных id по популярности (частота появления в просмотренных),
 сортировка просмотренных id по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

### Работа на обучающей выборке

In [364]:
train = pd.read_csv('coursera_sessions_train.txt', header=-1, sep=';', lineterminator='\n', names=['watch','buy'])

In [365]:
train.fillna('-1', inplace=True)

In [366]:
train.head(8)

,watch,buy
0,"0,1,2,3,4,5",-1
1,"9,10,11,9,11,12,9,11",-1
2,"16,17,18,19,20,21",-1
3,"24,25,26,27,24",-1
4,"34,35,36,34,37,35,36,37,38,39,38,39",-1
5,42,-1
6,"47,48,49",-1
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"


In [467]:
Train_w = [x.split(',') for x in train.watch]
Train_b = [x.split(',') for x in train.buy]

In [468]:
print(Train_w[7])
print(Train_b[7])

['59', '60', '61', '62', '60', '63', '64', '65', '66', '61', '67', '68', '67']
['67', '60', '63']


### 1

In [497]:
# Склеиваем сессии просмотров
Train_W=[]
for x in Train_w:
    Train_W += x

# частоты товаров по всем сессиям
C_w = Counter(Train_W)
# Популярность - сортировка частот по ключу с запоминанием порядка
Popularity = OrderedDict(sorted(C_w.items(), key=lambda t: t[1], reverse=True))

In [485]:
n = 0
Average_P_5=0
Average_R_5=0
Average_P_1=0
Average_R_1=0

for i in range(len(Train_w)):
    if Train_b[i]==Train_b[0]: continue
    
    n += 1

    # частоты для товаров из i сессии
    P = [Popularity[k] for k in M(Train_w[i])]

    # сшивка товаров из i сессии и их частоты (уникальные - множества)
    A = map(lambda x,y: [x,y], M(Train_w[i]),P)

    # Топ-5 по популярности в сессии
    Top_5 = sorted(A, key=lambda t: t[1], reverse=True)[:5]

    # количество пересечений просмотров и покупок по топу (посредсвтом множеств)
    K_top5 = len(set(Train_b[i]).intersection(set(np.array(Top_5)[:,0])))

    # был ли куплен самый топовый товар - первый в топе
    K_top1 = Train_b[i].count(Top_5[0][0]) # есть ли в списке значение sorted_w[0]
    
    
    
    # Precision@5 (количество покупок по рекомендации/число рекомендаций)
    Precision_5 = K_top5/5.

    # Recall@5 (количество покупок по рекомендации/число покупок)
    Recall_5 = K_top5/float(len(Train_b[i]))

    # Precision@1 
    Precision_1 = K_top1/1.

    # Recall@1
    Recall_1 = K_top1/float(len(Train_b[i]))

    # определяем суммы для средних
    Average_P_5 += Precision_5
    Average_R_5 += Recall_5
    Average_P_1 += Precision_1
    Average_R_1 += Recall_1

In [486]:
Average_P_5 = Average_P_5/float(n)
Average_R_5 = Average_R_5/float(n)
Average_P_1 = Average_P_1/float(n)
Average_R_1 = Average_R_1/float(n)

In [487]:
print(Average_R_1, Average_P_1, Average_R_5, Average_P_5)

(0.4426343165949593, 0.5121951219512195, 0.8246918247126122, 0.21252771618625918)


In [488]:
save_answers("0.44 0.51 0.82 0.21",'1.txt')

### 3

In [489]:
# Склеиваем сессии просмотров
Train_B=[]
for x in Train_b:
    if x==Train_b[0]: continue
    Train_B += x

# частоты товаров по всем сессиям
C_b = Counter(Train_B)
# Популярность - сортировка частот по ключу с запоминанием порядка
Popularity2 = OrderedDict(sorted(C_b.items(), key=lambda t: t[1], reverse=True))

In [490]:
n = 0
Average_P_2=0
Average_R_2=0
Average_P_1=0
Average_R_1=0

for i in range(len(Train_b)):
    if Train_b[i]==Train_b[0]: continue
    
    n += 1

    # частоты для товаров из i сессии
    P=[]
    for k in M(Train_w[i]):
        Popularity2.setdefault(k,0)
        P.append(Popularity2[k])

    # сшивка товаров из i сессии и их частоты (уникальные - множества)
    A = map(lambda x,y: [x,y], M(Train_w[i]),P)

    # Топ-5 по популярности в сессии
    Top_5 = sorted(A, key=lambda t: t[1], reverse=True)[:5]

    # количество пересечений просмотров и покупок по топу (посредсвтом множеств)
    K_top5 = len(set(Train_b[i]).intersection(set(np.array(Top_5)[:,0])))

    # был ли куплен самый топовый товар - первый в топе
    K_top1 = Train_b[i].count(Top_5[0][0]) # есть ли в списке значение sorted_w[0]
    
    
    # Precision@5 (количество покупок по рекомендации/число рекомендаций)
    Precision_5 = K_top5/5.

    # Recall@5 (количество покупок по рекомендации/число покупок)
    Recall_5 = K_top5/float(len(Train_b[i]))

    # Precision@1 
    Precision_1 = K_top1/1.

    # Recall@1
    Recall_1 = K_top1/float(len(Train_b[i]))

    # определяем суммы для средних
    Average_P_5 += Precision_5
    Average_R_5 += Recall_5
    Average_P_1 += Precision_1
    Average_R_1 += Recall_1

In [491]:
Average_P_5 = Average_P_5/float(n)
Average_R_5 = Average_R_5/float(n)
Average_P_1 = Average_P_1/float(n)
Average_R_1 = Average_R_1/float(n)

In [492]:
print(Average_R_1, Average_P_1, Average_R_5, Average_P_5)

(0.6884494924267653, 0.8037694013303769, 0.9265358755450274, 0.25260879371292183)


In [493]:
save_answers("0.69 0.80 0.93 0.25",'3.txt')

### Переходим на тестовую выборку

In [494]:
test = pd.read_csv('coursera_sessions_test.txt', header=-1, sep=';', lineterminator='\n', names=['watch','buy'])
test.fillna('-1', inplace=True)
Test_w = [x.split(',') for x in test.watch]
Test_b = [x.split(',') for x in test.buy]

In [495]:
test.head(3)

,watch,buy
0,"6,7,8",-1
1,"13,14,15",-1
2,"22,23",-1


### 2

In [498]:
n = 0
Average_P_5=0
Average_R_5=0
Average_P_1=0
Average_R_1=0

for i in range(len(Test_w)):
    if Test_b[i]==Test_b[0]: continue
    
    n += 1

    # частоты для товаров из i сессии
    P=[]
    for k in M(Test_w[i]):
        Popularity.setdefault(k,0)
        P.append(Popularity[k])
    
    
    # сшивка товаров из i сессии и их частоты (уникальные - множества)
    A = map(lambda x,y: [x,y], M(Test_w[i]),P)

    # Топ-5 по популярности в сессии
    Top_5 = sorted(A, key=lambda t: t[1], reverse=True)[:5]

    # количество пересечений просмотров и покупок по топу (посредсвтом множеств)
    K_top5 = len(set(Test_b[i]).intersection(set(np.array(Top_5)[:,0])))

    # был ли куплен самый топовый товар - первый в топе
    K_top1 = Test_b[i].count(Top_5[0][0]) # есть ли в списке значение sorted_w[0]
    
    
    
    # Precision@5 (количество покупок по рекомендации/число рекомендаций)
    Precision_5 = K_top5/5.

    # Recall@5 (количество покупок по рекомендации/число покупок)
    Recall_5 = K_top5/float(len(Test_b[i]))

    # Precision@1 
    Precision_1 = K_top1/1.

    # Recall@1
    Recall_1 = K_top1/float(len(Test_b[i]))

    # определяем суммы для средних
    Average_P_5 += Precision_5
    Average_R_5 += Recall_5
    Average_P_1 += Precision_1
    Average_R_1 += Recall_1

In [499]:
Average_P_5 = Average_P_5/float(n)
Average_R_5 = Average_R_5/float(n)
Average_P_1 = Average_P_1/float(n)
Average_R_1 = Average_R_1/float(n)

In [500]:
print(Average_R_1, Average_P_1, Average_R_5, Average_P_5)

(0.41733266203252534, 0.48130968622100956, 0.8000340663538579, 0.2037653478854079)


In [501]:
save_answers("0.42 0.48 0.80 0.20",'2.txt')

### 4

In [502]:
n = 0
Average_P_5=0
Average_R_5=0
Average_P_1=0
Average_R_1=0

for i in range(len(Test_w)):
    if Test_b[i]==Test_b[0]: continue
    
    n += 1

    # частоты для товаров из i сессии
    P=[]
    for k in M(Test_w[i]):
        Popularity2.setdefault(k,0)
        P.append(Popularity2[k])
    
    
    # сшивка товаров из i сессии и их частоты (уникальные - множества)
    A = map(lambda x,y: [x,y], M(Test_w[i]),P)

    # Топ-5 по популярности в сессии
    Top_5 = sorted(A, key=lambda t: t[1], reverse=True)[:5]

    # количество пересечений просмотров и покупок по топу (посредсвтом множеств)
    K_top5 = len(set(Test_b[i]).intersection(set(np.array(Top_5)[:,0])))

    # был ли куплен самый топовый товар - первый в топе
    K_top1 = Test_b[i].count(Top_5[0][0]) # есть ли в списке значение sorted_w[0]
    
    
    
    # Precision@5 (количество покупок по рекомендации/число рекомендаций)
    Precision_5 = K_top5/5.

    # Recall@5 (количество покупок по рекомендации/число покупок)
    Recall_5 = K_top5/float(len(Test_b[i]))

    # Precision@1 
    Precision_1 = K_top1/1.

    # Recall@1
    Recall_1 = K_top1/float(len(Test_b[i]))

    # определяем суммы для средних
    Average_P_5 += Precision_5
    Average_R_5 += Recall_5
    Average_P_1 += Precision_1
    Average_R_1 += Recall_1

In [503]:
Average_P_5 = Average_P_5/float(n)
Average_R_5 = Average_R_5/float(n)
Average_P_1 = Average_P_1/float(n)
Average_R_1 = Average_R_1/float(n)

In [504]:
print(Average_R_1, Average_P_1, Average_R_5, Average_P_5)

(0.4606201666660294, 0.5276944065484311, 0.8201874337490194, 0.21009549795362173)


In [505]:
save_answers("0.46 0.53 0.82 0.21",'4.txt')